# Nibiru Spots (Swaps) Queries and Transactions
Examples of executing asset swaps.

### 1. Install Nibiru package

In [57]:
import time
!pip install nibiru

  Using cached importlib_metadata-5.2.0-py3-none-any.whl (21 kB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Using cached pre_commit-2.21.0-py2.py3-none-any.whl (201 kB)
  Using cached python_dotenv-0.21.1-py3-none-any.whl (19 kB)
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.0
    Uninstalling python-dotenv-1.0.0:
      Successfully uninstalled python-dotenv-1.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packaging-23.2
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.8.0
    Uninstalling importlib-metadata-6.8.0:
      Successfully uninstalled importlib-metadata-6.8.0
  Attempting uninstall: pre-commit
    Found existing installation: pre-commit 3.4.0
    Uninstalling pre-commit-3.4.0:
      Successfully uninstalled pre-commit-3.4.0


### 2. Create chain client and run queries
Using testnet-3 (aka itn-3) endpoint run queries.

In [58]:
import json
from nibiru import Network, ChainClient, Msg, Coin, TxConfig
from nibiru.exceptions import QueryError

client = ChainClient(network=Network.testnet(3))

In [59]:
# Shortcut to print responses
def print_json(obj):
    print(json.dumps(obj, indent=2))

In [60]:
# Query spot params (fees and whitelisted assets)
spot_params = client.query.spot.params()
print_json(spot_params)

{
  "startingPoolNumber": "1",
  "poolCreationFee": [
    {
      "denom": "unibi",
      "amount": "1000000000"
    }
  ],
  "whitelistedAsset": [
    "unibi",
    "uusdc",
    "unusd",
    "uusdt"
  ]
}


In [61]:
# Query spot pools
pools = client.query.spot.pools()
print_json(pools)

[
  {
    "id": "1",
    "address": "nibi1w00c7pqkr5z7ptewg5z87j2ncvxd88w43ug679",
    "poolParams": {
      "swapFee": "10000000000000000",
      "exitFee": "10000000000000000",
      "A": "10",
      "poolType": "STABLESWAP"
    },
    "poolAssets": [
      {
        "token": {
          "denom": "unusd",
          "amount": "343916448"
        },
        "weight": "1073741824"
      },
      {
        "token": {
          "denom": "uusdt",
          "amount": "6745263529"
        },
        "weight": "1073741824"
      }
    ],
    "totalWeight": "2147483648",
    "totalShares": {
      "denom": "nibiru/pool/1",
      "amount": "311755152668458412094"
    }
  },
  {
    "id": "2",
    "address": "nibi1vm70ju9n2laspspsh6fs2hu50z3qzf4m2emr0p",
    "poolParams": {
      "swapFee": "10000000000000000",
      "exitFee": "10000000000000000",
      "A": "0"
    },
    "poolAssets": [
      {
        "token": {
          "denom": "unibi",
          "amount": "443755190"
        },
        "

### 3. Authenticate to run transactions
Assuming you have your wallet mnemonic key (or private key) and some test tokens from the faucet.
See: https://app.nibiru.fi/faucet

In [62]:
mnemonic = "exclude rural knife asset egg seminar carry control thought stock topic lens embark about huge obvious reason language ripple clean cousin library punch renew"

# Let's set high gas multiplier to avoid out of gas errors. Could be customized per tx.
tx_config = TxConfig(gas_multiplier=5)
client = ChainClient(Network.testnet(3), tx_config=tx_config)
client.authenticate(mnemonic=mnemonic)

# Balances check
balances = client.query.get_bank_balances(client.address)
print_json(balances["balances"])

[
  {
    "denom": "nibiru/pool/1",
    "amount": 1012681954724
  },
  {
    "denom": "unibi",
    "amount": 99999933066444
  },
  {
    "denom": "unusd",
    "amount": 999999999999999692
  },
  {
    "denom": "uusdt",
    "amount": 10000000000000352
  }
]


### 4. Run Spot transactions
Swap tokens:

In [63]:
pool_id = 1  # select the pool from the query above
token_in = "unusd"  # check that token is in the pool
token_out = "uusdt" # check that token is in the pool

output = client.tx.execute_msgs(
    Msg.spot.swap(
        pool_id=pool_id,
        token_in=Coin(10, token_in),
        token_out_denom=token_out,
    ),
    wait_for_tx_resp=True,  # wait for block and get tx response
)
# Expected tx response code 0
print(f"tx response Code: {output['code']}")
if output["code"] == 0:
    balances = client.query.get_bank_balances(client.address)
    print_json(balances["balances"])

tx response Code: 0
[
  {
    "denom": "nibiru/pool/1",
    "amount": 1012681954724
  },
  {
    "denom": "unibi",
    "amount": 99999932972226
  },
  {
    "denom": "unusd",
    "amount": 999999999999999682
  },
  {
    "denom": "uusdt",
    "amount": 10000000000000378
  }
]


Join spot pool. After joining the pool you will have a pool shares in your balance with asset name like "nibiru/pool/1".

In [64]:
output = client.tx.execute_msgs(
    Msg.spot.join_pool(
        pool_id=pool_id,
        tokens=[Coin(10, token_in), Coin(5, token_out)]
    ),
    wait_for_tx_resp=True,
)
# Expected tx response code 0
print(f"tx response code: {output['code']}")
if output["code"] == 0:
    balance = client.query.get_bank_balance(client.address, f"nibiru/pool/{pool_id}")
    print_json(balance["balance"])
    total_shares_amount = balance["balance"]["amount"]

tx response code: 0
{
  "denom": "nibiru/pool/1",
  "amount": 2353780420866
}


Exit pool. Pool shares will disappear from balances list.

In [65]:
time.sleep(5)  # Sometimes it needs several seconds to allow pool exit
output = client.tx.execute_msgs(
    Msg.spot.exit_pool(
        pool_id=pool_id,
        pool_shares=Coin(total_shares_amount, f"nibiru/pool/{pool_id}")
    ),
    wait_for_tx_resp=True,
)
# Expected tx response code 0
print(f"tx response code: {output['code']}")
if output["code"] == 0:
    balances = client.query.get_bank_balances(client.address)
    print_json(balances["balances"])

tx response code: 0
[
  {
    "denom": "unibi",
    "amount": 99999932733366
  },
  {
    "denom": "unusd",
    "amount": 999999999999999674
  },
  {
    "denom": "uusdt",
    "amount": 10000000000000423
  }
]
